In [1]:
import os
import re
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

/home/huisyuan/anaconda3/envs/tf_env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = pd.read_csv("norm_data__non_log.txt",sep='\t')
#data

In [3]:
# Extract labels

def extract_label(list): 
    number = '[0-9]'
    symbol = '_'
    head = 'Sample'
    list = [re.sub(number, '', i) for i in list] 
    list = [re.sub(symbol, '', i) for i in list] 
    list = [re.sub(head, '', i) for i in list] 
    return list

labels = list(data)
labels = extract_label(labels)
labels = np.ravel(labels)

data=data.T

mapping = {'Non-LCa':-1,'LCa':1}
data["targets"] = labels
data["targets"] = data["targets"].map(mapping)
#data

#X = data.iloc[:,0:1183].as_matrix()
#y = data["targets"].as_matrix()
#print(X.shape)
#print(y.shape)


In [6]:
# Log and Scale Features in [0,1]

scaler = MinMaxScaler() 

X = data.iloc[:,0:1183]
X = np.log(X)
data_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

#scale targets too
#data_scaled = pd.DataFrame(scaler.fit_transform(data), columns=data.columns) 

data_scaled
X = data_scaled.as_matrix()
y = data["targets"].as_matrix
# type(X)
# X.shape

/home/huisyuan/anaconda3/envs/tf_env/lib/python3.5/site-packages/ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]


In [ ]:
# X = data.iloc[:,0:1183].as_matrix()
# y = data["targets"].as_matrix()
# #print(X.shape)
# #print(y.shape)

In [ ]:
# pdf with KDE

# from sklearn.neighbors import KernelDensity
# kde = KernelDensity(kernel='gaussian', bandwidth=0.25).fit(X)
# kde.sample(3)

In [ ]:
# score_samples returns the log of the probability density

# print(X[11])
# print(X[200])

# print(kde.score_samples(X[11].reshape(1,-1)),
#       kde.score_samples(X[180].reshape(1,-1)),
#       kde.score_samples(X[18].reshape(1,-1)),
#       kde.score_samples(X[10].reshape(1,-1)),
#       kde.score_samples(X[100].reshape(1,-1)),
#       kde.score_samples(X[1000].reshape(1,-1)),)

# logprob=kde.score_samples(X)
# print(logprob)
# print(np.exp(logprob))

In [ ]:
# Try one pair

# i = 1000
# j = 100

# from sklearn.neighbors import KernelDensity

# xgrid = np.linspace(0, 1, 1000)
# kde_i = KernelDensity(kernel='gaussian', bandwidth=0.4).fit(X[:,[i]].reshape(-1,1))
# kde_j = KernelDensity(kernel='gaussian', bandwidth=0.4).fit(X[:,[j]].reshape(-1,1))
# sample_prob_i = kde_i.score_samples(xgrid.reshape(-1,1))
# sample_prob_j = kde_j.score_samples(xgrid.reshape(-1,1))

# #diff = np.sum((np.exp(sample_prob_i)-np.exp(sample_prob_j))**2)
# diff = np.sum(((sample_prob_i)-(sample_prob_j))**2)
# print(diff)


# print(np.exp(sample_prob_i))
# print(np.exp(sample_prob_j))
# print(sample_prob_i)
# print(sample_prob_j)

In [7]:
# All pairs

from sklearn.neighbors import KernelDensity

xgrid = np.linspace(0, 1, 10)
#sim_table = pd.DataFrame()
num_SimilarFeature = []

for i in range(0,1183):
    diff_i = []
    kde_i = KernelDensity(kernel='gaussian', bandwidth=0.4).fit(X[:,[i]].reshape(-1,1))
    sample_prob_i = kde_i.score_samples(xgrid.reshape(-1,1))
    
    for j in range(0,1183):
        #if j!=i: 

        kde_j = KernelDensity(kernel='gaussian', bandwidth=0.4).fit(X[:,[j]].reshape(-1,1))
        sample_prob_j = kde_j.score_samples(xgrid.reshape(-1,1))
        diff = np.sum(((sample_prob_i)-(sample_prob_j))**2)   
#         print(i,j,diff)
        diff_i.append(diff)
#     print(diff_i)
#     the SimilarFeature includes itself
    
    num_SimilarFeature_i = np.sum(1 for s in diff_i if s<=1)
    num_SimilarFeature.append(num_SimilarFeature_i)
print(num_SimilarFeature)
#     print(num_SimilarFeature_i)    
#     sim_table[i] = diff_i
# sim_table         

[651, 585, 299, 489, 534, 545, 667, 564, 573, 462, 680, 698, 466, 444, 395, 672, 532, 363, 404, 507, 616, 578, 669, 122, 645, 655, 367, 438, 473, 656, 303, 624, 471, 645, 341, 583, 689, 104, 334, 660, 395, 596, 438, 193, 592, 648, 643, 28, 361, 676, 679, 682, 616, 299, 660, 664, 653, 136, 682, 404, 680, 488, 527, 684, 565, 682, 642, 683, 673, 550, 352, 481, 672, 672, 398, 388, 670, 627, 668, 113, 670, 151, 638, 682, 422, 650, 519, 688, 203, 494, 532, 687, 425, 623, 617, 388, 673, 672, 466, 510, 599, 545, 669, 685, 631, 503, 366, 572, 563, 668, 337, 660, 292, 613, 630, 644, 694, 471, 567, 682, 671, 578, 388, 677, 451, 635, 444, 406, 683, 618, 693, 347, 677, 662, 493, 588, 203, 633, 615, 681, 672, 624, 503, 420, 595, 651, 698, 620, 575, 614, 404, 684, 679, 683, 671, 386, 414, 40, 671, 407, 682, 690, 655, 97, 178, 466, 388, 402, 651, 664, 673, 667, 530, 513, 683, 620, 545, 617, 686, 279, 673, 269, 297, 494, 684, 459, 617, 680, 545, 666, 684, 176, 668, 685, 342, 426, 99, 262, 426, 682, 686

In [8]:
with open('sim_density_logm.txt', 'w') as f:
    for item in num_SimilarFeature:
        f.write("%s\n" % item)

f.close()

In [26]:
# Extract top-k distinct features 

k1 = [i for i in range(len(num_SimilarFeature)) if num_SimilarFeature[i] < 71]
print('best',len(k1),':\n',k1)

k2 = [i for i in range(len(num_SimilarFeature)) if num_SimilarFeature[i] < 80]
print('best',len(k2),':\n',k2)

k3 = [i for i in range(len(num_SimilarFeature)) if num_SimilarFeature[i] < 90]
print('best',len(k3),':\n',k3)

with open('sim_density_log_top33.txt', 'w') as f:
    for item in k1:
        f.write("%s\n" % item)
f.close()
        
with open('sim_density_log_top40.txt', 'w') as f:
    for item in k2:
        f.write("%s\n" % item)
f.close()
        
with open('sim_density_log_top44.txt', 'w') as f:
    for item in k3:
        f.write("%s\n" % item)
f.close()

best 33 :
 [47, 157, 299, 303, 315, 335, 429, 439, 499, 509, 548, 597, 626, 673, 688, 694, 781, 811, 829, 852, 861, 869, 877, 933, 966, 981, 1011, 1046, 1072, 1104, 1139, 1157, 1171]
best 40 :
 [47, 157, 212, 299, 303, 315, 335, 429, 432, 436, 439, 478, 499, 509, 548, 597, 615, 626, 673, 688, 694, 730, 781, 811, 829, 852, 861, 869, 877, 901, 933, 966, 981, 1011, 1046, 1072, 1104, 1139, 1157, 1171]
best 44 :
 [47, 157, 212, 272, 299, 303, 315, 335, 429, 432, 436, 439, 478, 499, 509, 548, 597, 615, 626, 673, 688, 694, 730, 781, 811, 829, 840, 852, 861, 869, 877, 901, 905, 933, 966, 981, 1011, 1046, 1072, 1104, 1139, 1157, 1171, 1179]


In [ ]:
# Reduce Dimension

# k2 = []
# with open("sim_density_top30.txt") as f:
#     for line in f:
#         k2.append(line)
        
# k2 = [float(i) for i in k2]        
        
# X_new = data.iloc[:,k2]
# print(X_new.shape)